In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import GPy, scipy

## Bayesian Linear Regression 

In [ ]:
def phiGen(p):
    return lambda x: np.array([x**i for i in range(p+1)]).T

p = 3
phi = phi_gen(p)

In [ ]:
x = np.linspace(-1,1)

plt.plot(x, phi(x));

plt.axhline(0, color='black')
plt.axvline(0, color='black')

In [ ]:
beta_cov = np.eye(p+1)

f_cov = np.dot(phi(x), np.dot(beta_cov, phi(x).T))

plt.imshow(f_cov)
plt.colorbar()

In [ ]:
f = scipy.stats.multivariate_normal.rvs(np.zeros(50), f_cov, size=10).T

plt.plot(x,f)

In [ ]:
def fdist(x, phi, beta_cov, sigma = 0.0):
    f_cov = np.dot(phi(x), np.dot(beta_cov, phi(x).T)) + np.eye(x.shape[0])*sigma
    
    jit = 1e-6
    excp = None
    #return scipy.stats.multivariate_normal(np.zeros(x.shape[0]), f_cov+jit*np.eye(x.shape[0]))
    while jit<1e1:
        try:
            #return scipy.stats.multivariate_normal(np.zeros(x.shape[0]), f_cov)
            return scipy.stats.multivariate_normal(np.zeros(x.shape[0]), f_cov+jit*np.eye(x.shape[0]))
        except np.linalg.LinAlgError as e:
            jit = jit*10
            excp = e
    
    raise excp

In [ ]:
p = 4
xsamp = np.random.choice(x, size=3, replace=False)
fsamp = fdist(xsamp, phi_gen(p), np.eye(p+1)).rvs()

plt.scatter(xsamp, fsamp)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

In [ ]:
kzx = np.dot(phi_gen(p)(x), np.dot(np.eye(p+1), phi_gen(p)(xsamp).T))
kzz = np.dot(phi_gen(p)(x), np.dot(np.eye(p+1), phi_gen(p)(x).T))
kxx = np.dot(phi_gen(p)(xsamp), np.dot(np.eye(p+1), phi_gen(p)(xsamp).T))

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
cov_pred[cov_pred<0] = 0
std_pred = np.sqrt(np.diag(cov_pred))

# plt.subplot(121)
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

Add observation noise

In [ ]:
x = np.linspace(-1,1)

sigma = 5e-0
p = 2
phi = phiGen(p)
xsamp = np.random.uniform(x.min(), x.max(), size=100)
fsamp = fdist(xsamp, phi_gen(p), np.eye(p+1), sigma=sigma).rvs()

plt.scatter(xsamp, fsamp)
plt.axhline(0, color='black')
plt.axvline(0, color='black')

In [ ]:
kzx = np.dot(phi_gen(p)(x), np.dot(np.eye(p+1), phi_gen(p)(xsamp).T))
kzz = np.dot(phi_gen(p)(x), np.dot(np.eye(p+1), phi_gen(p)(x).T))
kxx = np.dot(phi_gen(p)(xsamp), np.dot(np.eye(p+1), phi_gen(p)(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
# cov_pred[cov_pred<0] = 0
std_pred = np.sqrt(np.diag(cov_pred))

# plt.subplot(121)
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

In [ ]:
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.plot(x, scipy.stats.multivariate_normal.rvs(mu_pred, cov_pred, size=10).T,alpha=.5);

compute sub-components

In [ ]:
psi = [0] + [1] + [0]*(p-1)
psi*phi(x)

In [ ]:
kzx = np.dot(psi*phi(x), np.dot(np.eye(p+1), phi(xsamp).T))
kzz = np.dot(psi*phi(x), np.dot(np.eye(p+1), (psi*phi(x)).T))
kxx = np.dot(phi(xsamp), np.dot(np.eye(p+1), phi(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
# cov_pred[cov_pred<0] = 0
std_pred = np.sqrt(np.diag(cov_pred))

# plt.subplot(121)
plt.plot(x, mu_pred)
plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp)

In [ ]:
for i in range(p+1):
    psi = [0]*(p+1)
    psi[i] = 1
    
    kzx = np.dot(psi*phi(x), np.dot(np.eye(p+1), phi(xsamp).T))
    kzz = np.dot(psi*phi(x), np.dot(np.eye(p+1), (psi*phi(x)).T))
    kxx = np.dot(phi(xsamp), np.dot(np.eye(p+1), phi(xsamp).T)) + sigma*np.eye(xsamp.shape[0])

    mu_pred = np.dot(kzx, np.dot(np.linalg.inv(kxx), fsamp))
    cov_pred = kzz - np.dot(kzx, np.dot(np.linalg.inv(kxx), kzx.T))
    std_pred = np.sqrt(np.diag(cov_pred))

    plt.plot(x, mu_pred, label='$\phi_%d$'%i)
    plt.fill_between(x, mu_pred-2*std_pred, mu_pred+2*std_pred, alpha=.1)
plt.scatter(xsamp, fsamp,c='k',alpha=.5)
plt.legend()

In [ ]:
# parameter inference
ko = np.eye(p+1)
kfo = np.dot(phi(xsamp), ko)
kf = phi(xsamp).dot(ko).dot(phi(xsamp).T) + sigma*np.eye(xsamp.shape[0])

mu_pred = kfo.T.dot(np.linalg.inv(kf)).dot(fsamp)
cov_pred = ko - kfo.T.dot(np.linalg.inv(kf)).dot(kfo)

plt.errorbar(range(p+1), mu_pred, yerr=np.sqrt(cov_pred.diagonal()))

### Compare to linear regression 

In [ ]:
omega_hat = np.linalg.inv(phi(xsamp).T.dot(phi(xsamp))).dot(phi(xsamp).T).dot(fsamp)

plt.errorbar(range(p+1), mu_pred, yerr=np.sqrt(cov_pred.diagonal()))
plt.scatter(range(p+1), omega_hat)

## GP basics

In [ ]:
x = np.linspace(-3,3, 101)[:,None]
kern = GPy.kern.RBF(1)

plt.plot(x[x.shape[0]/2,0] - x, kern.K(x)[x.shape[0]/2,:])
plt.xlabel("x-x'", fontsize=20)

plt.figure()
plt.imshow(kern.K(x))
plt.colorbar()

In [ ]:
s1 = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), kern.K(x), size=3).T

plt.plot(x, s1, c='C0');

In [ ]:
# kern2 = GPy.kern.RBF(1, variance=1e0, lengthscale=1e0)
kern2 = GPy.kern.RBF(1, variance=1e0, lengthscale=3e0)
# kern2 = GPy.kern.RBF(1, variance=1e0, lengthscale=5e-1)
# kern2 = GPy.kern.RBF(1, variance=1e-1, lengthscale=1e0)
# kern2 = GPy.kern.RBF(1, variance=1e1, lengthscale=1e0)

s2 = scipy.stats.multivariate_normal.rvs(np.zeros(x.shape[0]), kern2.K(x), size=3).T

plt.figure(figsize=(10,6))

plt.subplot(121)
plt.plot(x[x.shape[0]/2,0] - x, kern.K(x)[x.shape[0]/2,:])
plt.plot(x[x.shape[0]/2,0] - x, kern2.K(x)[x.shape[0]/2,:])
plt.xlabel("x-x'", fontsize=20)

plt.subplot(122)
plt.plot(x, s1, c='C0');
plt.plot(x, s2, c='C1');